# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,talnakh,69.4865,88.3972,-24.77,99,100,4.01,RU,1701895499
1,1,port-aux-francais,-49.3500,70.2167,4.75,85,100,8.98,TF,1701895499
2,2,campbellton,48.0075,-66.6727,-6.75,78,10,3.72,CA,1701895499
3,3,anadyr,64.7500,177.4833,-20.56,76,100,15.00,RU,1701895499
4,4,edinburgh of the seven seas,-37.0676,-12.3116,14.35,76,10,8.10,SH,1701895499


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display
import folium

# Create a map object
m = folium.Map(location=[0, 0], zoom_start=2)  # Starting at a central point on the globe

# Add points for each city in the DataFrame
for index, row in city_data_df.iterrows():
    # Create a circle marker with size based on humidity
    folium.CircleMarker(
        location=[row['Lat'], row['Lng']],  # Latitude and Longitude
        radius=row['Humidity'] * 0.1,  # Adjust the size to be suitable for the map, you may need to scale the humidity value
        popup=f"{row['City']}, {row['Country']} - Humidity: {row['Humidity']}%",
        color='blue',
        fill=True,
        fill_color='blue'
    ).add_to(m)

# Display the map
m


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_narrow = city_data_df[(city_data_df['Max Temp'] < 27) &
                             (city_data_df['Max Temp'] > 21) &
                             (city_data_df['Wind Speed'] < 10) &
                             (city_data_df['Cloudiness'] == 0) &
                             (city_data_df['Humidity'] < 30)]
city_data_narrow = city_data_narrow.dropna() # Drop any rows with null values

# Display the count of rows that match the criteria
city_data_narrow.count()


City_ID       3
City          3
Lat           3
Lng           3
Max Temp      3
Humidity      3
Cloudiness    3
Wind Speed    3
Country       3
Date          3
dtype: int64

### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
import pandas as pd

# Assuming 'city_data_df' is your existing DataFrame and it contains columns for city, country, latitude (Lat), longitude (Lng), and humidity
# Select the required columns and create a new DataFrame

hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Optionally, you can rename the columns for clarity, for example:
hotel_df.columns = ['City', 'Country', 'Coordinates (Lat)', 'Coordinates (Lng)', 'Humidity']

# Display the first few rows of the new DataFrame
print(hotel_df.head())


                          City Country  Coordinates (Lat)  Coordinates (Lng)  \
0                      talnakh      RU            69.4865            88.3972   
1            port-aux-francais      TF           -49.3500            70.2167   
2                  campbellton      CA            48.0075           -66.6727   
3                       anadyr      RU            64.7500           177.4833   
4  edinburgh of the seven seas      SH           -37.0676           -12.3116   

   Humidity  
0        99  
1        85  
2        78  
3        76  
4        76  


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000  # 10,000 meters
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat, lng = row['Coordinates (Lat)'], row['Coordinates (Lng)']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()


Starting hotel search
talnakh - nearest hotel: Талнах
port-aux-francais - nearest hotel: Keravel
campbellton - nearest hotel: Campbellton City Center Mall
anadyr - nearest hotel: Гостевой дом
edinburgh of the seven seas - nearest hotel: No hotel found
jamestown - nearest hotel: DoubleTree Jamestown
pisco - nearest hotel: La Portada
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
san luis de la loma - nearest hotel: No hotel found
kapuskasing - nearest hotel: Travelodge
rabo de peixe - nearest hotel: Pico do Refugio
darwin - nearest hotel: Mantra Pantanas Darwin
nova sintra - nearest hotel: Residência Ka Dencho
albany - nearest hotel: No hotel found
invercargill - nearest hotel: The Grand
adamstown - nearest hotel: No hotel found
ta`u - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
east london - nearest hotel: No hotel found
hermanus - nearest hotel: Aloe guest house
khovd - nearest hotel: Khovd hotel
kerikeri - nearest hotel: Avalon Resort
codrin

pandan - nearest hotel: No hotel found
fangshan - nearest hotel: 速8酒店
luena - nearest hotel: No hotel found
puerto lempira - nearest hotel: Hotel Yu Baiwan
bereznik - nearest hotel: Отель Прибрежный
bam - nearest hotel: هتل جهانگردی بم
craig - nearest hotel: Quality Inn & Suites
general luna - nearest hotel: No hotel found
hovd - nearest hotel: Khovd hotel
stanley - nearest hotel: Hotel 52
naraina - nearest hotel: No hotel found
udachny - nearest hotel: Вилюй
dean funes - nearest hotel: Hospedaje Meli
pontarlier - nearest hotel: L'atelier de Donat
hamilton - nearest hotel: North Vista Manor
guatire - nearest hotel: Grand Hotel
bandarbeyla - nearest hotel: No hotel found
todos santos - nearest hotel: Hotel Casa Tota
zag - nearest hotel: No hotel found
ancud - nearest hotel: Cabañas
akjoujt - nearest hotel: فندق فاطمة
lamu - nearest hotel: Hotel Archipelago villa bed and breakfast
pangoa - nearest hotel: No hotel found
magadan - nearest hotel: Гостиница «ВМ-Центр»
aldinga beach - nearest

tukrah - nearest hotel: No hotel found
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
korfez - nearest hotel: Gümüşel
askiz - nearest hotel: No hotel found
marsh harbour - nearest hotel: Lofty Fig Villas
envira - nearest hotel: No hotel found
ayorou - nearest hotel: Hotel Aminokal
kyaikto - nearest hotel: Shwe Hintar
sisimiut - nearest hotel: Sømandshjemmet
nizhnevartovsk - nearest hotel: Самотлор
gandi - nearest hotel: No hotel found
dalai - nearest hotel: No hotel found
asaita - nearest hotel: ናርዶስ ሆቴል
tshikapa - nearest hotel: Hôtel TSHAMUNTU
sitka - nearest hotel: Aspen Suites Hotel
monzon - nearest hotel: No hotel found
praia da vitoria - nearest hotel: Salles
arraial do cabo - nearest hotel: No hotel found
buta - nearest hotel: Hôtel Rubi
kharan - nearest hotel: No hotel found
kulhudhuffushi - nearest hotel: No hotel found
al bawiti - nearest hotel: Old Oasis Hotel
belawan - nearest hotel: Danau Toba International Cottage Belawan
prosser - nearest hotel: Be

,City,Country,Coordinates (Lat),Coordinates (Lng),Humidity,Hotel Name
0,talnakh,RU,69.4865,88.3972,99,Талнах
1,port-aux-francais,TF,-49.3500,70.2167,85,Keravel
2,campbellton,CA,48.0075,-66.6727,78,Campbellton City Center Mall
3,anadyr,RU,64.7500,177.4833,76,Гостевой дом
4,edinburgh of the seven seas,SH,-37.0676,-12.3116,76,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Create a map object
m = folium.Map(location=[0, 0], zoom_start=2)  # Starting at a central point on the globe

# Add points for each city in the DataFrame
for index, row in hotel_df.iterrows():
    # Create the popup message
    popup_message = f"City: {row['City']}, Country: {row['Country']}, Hotel: {row['Hotel Name']}"

    # Create a circle marker with size based on humidity
    folium.CircleMarker(
        location=[row['Coordinates (Lat)'], row['Coordinates (Lng)']],  # Latitude and Longitude
        radius=row['Humidity'] * 0.1, 
        popup=popup_message,
        color='blue',
        fill=True,
        fill_color='blue'
    ).add_to(m)

# Display the map
m
